In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
def analyze_financial_sentiment(news_text, tokenizer, model):
    inputs = tokenizer.encode_plus(news_text, return_tensors="pt", add_special_tokens=True, truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    sentiment_scores = torch.nn.functional.softmax(logits, dim=1).squeeze().tolist()
    return predicted_class, sentiment_scores

In [ ]:
def assess_equity_risk(sentiment_class):
    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    sentiment = sentiment_labels[sentiment_class]

    if sentiment == 'Positive':
        return "Low Risk"
    elif sentiment == 'Negative':
        return "High Risk"
    else:
        return "Medium Risk"

In [ ]:
def main():
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    dataset = pd.read_csv('Tesla.csv')
    
    # Add new columns 'Sentiment' and 'Sentiment Scores' to the dataset
    dataset['Sentiment'], dataset['Sentiment Scores'] = zip(*dataset['News'].apply(lambda x: analyze_financial_sentiment(x, tokenizer, model)))
    
    # Add a new column 'Risk Assessment' to the dataset
    dataset['Risk Assessment'] = dataset['Sentiment'].apply(lambda x: assess_equity_risk(x))
    
    pd.set_option('display.max_colwidth', None)  # To display complete input statements
    data1=print(dataset[['News', 'Sentiment', 'Risk Assessment']])
    
if __name__ == "__main__":
    main()